[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/multi-tenancy/multi-tenancy-example.ipynb)

# 🗂️ Multi-Tenancy in Weaviate

Welcome to this demo notebook! Here, we'll walk you through a small example showcasing the `Multi-Tenancy` function in Weaviate.
Multi-tenancy is a key feature in Weaviate, allowing for the efficient and secure management of data across multiple users or tenants.

## 📖 Further Reading:

- Explore the concept in depth in the [multi-tenancy blog post](https://weaviate.io/blog/multi-tenancy-vector-search).
- Dive into the technical details in our [Weaviate developer documentation](https://weaviate.io/developers/weaviate/manage-data/multi-tenancy#enable-multitenancy).

## Getting started
Before we dive in, there are a few preliminary steps:

1. Set Up a Weaviate Cluster: 
This notebook requires a working Weaviate cluster. If you don't have one, fret not! You can set up a free sandbox Weaviate cluster by following our [comprehensive guide](https://weaviate.io/developers/academy/zero_to_mvp/hello_weaviate/set_up).

2. Virtual Environment and Dependencies: 
To ensure smooth execution and prevent potential conflicts with your global Python environment, we recommend running the code in a virtual environment. Later in this notebook, we'll guide you through setting up this environment and installing the necessary dependencies.

With these points in mind, let's get started!

## Dependencies

Before proceeding with the notebook content, it's essential to set up an isolated Python environment. This helps avoid any potential package conflicts and ensures that you have a clean workspace.

### Virtual Environment Setup:

If you haven't created a virtual environment before, here's how you can do it:

Using `virtualenv`:

```bash
pip install virtualenv
python -m virtualenv venv
```

Using `venv` (built-in with Python 3.3+):

```bash
python -m venv venv
```

After creating the virtual environment, you need to activate it:

Windows:

```bash
.\venv\Scripts\activate
```
macOS and Linux:

```bash
source venv/bin/activate
```
### Installing Dependencies:

With the virtual environment active, run the following code to install all the required dependencies for this notebook:

In [1]:
# Uncomment & run the following line if you need to install the Weaviate client
# %pip install weaviate-client

## Connecting to Your Weaviate Cluster

To interact with our Weaviate cluster, we'll initialize a client object. Once set up, we'll retrieve the current schemas as a way to verify the connection. Since the cluster is newly created, we expect that no schemas will be present.

In [2]:
import weaviate

client = weaviate.connect_to_local()

# # If you are using a cloud (WCS) instance
#
# import weaviate
# import os
#
# client = weaviate.connect_to_wcs(
#     cluster_url="WEAVIATE-INSTANCE-URL",  # URL of your Weaviate instance
#     auth_credentials=weaviate.auth.AuthApiKey("AUTH-KEY"),  # (Optional) If the Weaviate instance requires authentication
#     headers={
#         "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY"),
#     }
# )

## Setting Up Multi-Tenancy in a Weaviate Collection

In Weaviate, multi-tenancy allows for multiple tenants to securely access and manage data within the same schema. Let's proceed to define a new collection that utilizes this feature:

### Define a Multi-Tenancy Enabled Collection:
We'll start by creating a collection named 'MultiTenancyCollection' with the multi-tenancy feature activated.

In [ ]:
from weaviate.classes.config import Configure

# Delete the collection if it already exists (for the sake of the example)
if(client.collections.exists("MyCollection")):
    client.collections.delete("MyCollection")

client.collections.create(
    name="MyCollection",
    multi_tenancy_config=Configure.multi_tenancy(enabled=True)
)

### Add Multiple Tenants to the Class:
After establishing the collection, we'll associate it with two tenants: 'tenantA' and 'tenantB'

In [4]:
from weaviate.classes.tenants import Tenant

my_collection = client.collections.get("MyCollection")  # Get the "MultiTenancyCollection" collection

my_collection.tenants.create([
    Tenant(name='tenantA'),
    Tenant(name='tenantB')
]) # add two tenants

## Fetching Tenants from a Weaviate Collection

To view the tenants associated with a specific collection, we can retrieve a list of all the tenants linked to it. Let's do this for our previously created collection, 'MultiTenancyCollection':

In [ ]:
tenants = my_collection.tenants.get()
print(tenants)
assert len(tenants) == 2

## Assigning Data Objects to Specific Tenants

In Weaviate, data objects can be associated with specific tenants in a multi-tenancy enabled collection. Here, we will demonstrate how to create data objects and link them to their respective tenants:

In [6]:
# Define objects to insert into each tenant
data_objects = {
    "tenantA": ["This belongs to TenantA"],
    "tenantB": ["This belongs to TenantB", "This also belongs to TenantB"]
}

for tenant_name, tenant_obj in tenants.items():                 # Iterate through the tenants

    tenant_collection = my_collection.with_tenant(tenant_name)     # Tenant-specific collection

    for text_data in data_objects[tenant_name]:                       # Iterate through the objects for the tenant
        tenant_collection.data.insert(                              # Insert a data object into the tenant-specific collection
            properties={"text": text_data}
        )

## Performing Tenant-Specific Queries

By leveraging the multi-tenancy functionality, we can conduct queries that are specific to individual tenants. This enables us to fetch data solely associated with a designated tenant.

In [ ]:
for tenant_name, tenant_obj in tenants.items():                 # Iterate through the tenants

    tenant_collection = my_collection.with_tenant(tenant_name)     # Tenant-specific collection

    response = tenant_collection.query.fetch_objects()          # Fetch some objects from the tenant-specific collection
    print(f"Tenant {tenant_name} has {len(response.objects)} objects.")
    for obj in response.objects:
        print(obj.properties)

    assert len(response.objects) == len(data_objects[tenant_name])  # Check if the number of objects in the tenant-specific collection is equal to the number of objects inserted

## Removing Tenants from a Weaviate Collection

In situations where specific tenants are no longer required, Weaviate allows us to remove them from a collection. This action will only affect the specified tenants, leaving other associated tenants unaffected.

In [8]:
collection.tenants.remove(['tenantB', 'tenantX'])  # The tenants to be removed. `tenantX`` will be ignored as it does not exist.

assert len(collection.tenants.get()) == 1